### Install

In [5]:
! pip install -qq datasets==2.16.1 tiktoken langchain langchain_openai==0.0.7 requests==2.28.2 transformers==4.41.2 tqdm

### Imports

In [4]:
import json
import os
import getpass
import tiktoken
import pandas as pd
from datasets import Dataset, load_dataset
from tqdm import tqdm
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate
from langchain_openai import OpenAI
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser

### Load dataset

In [2]:
dataset = load_dataset("wikimedia/wikipedia", "20231101.en")

Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

### Selecting subset

In [6]:
len(dataset["train"])

6407814

In [68]:
NUM_SAMPLES = 10000
articles = dataset["train"][:NUM_SAMPLES]["text"]
articles = [x.split("\n")[0] for x in articles]

### Initialize OpenAI Environment

In [8]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

Enter your OpenAI API key: ········


In [9]:
llm = OpenAI()

### Set the output parser

In [12]:
class Classification(BaseModel):
    category: str = Field(description="Category of the article", default="Others")
parser = JsonOutputParser(pydantic_object=Classification)

### Create classification prompt

In [13]:
template = """Your task is to assess the article and categorize the article into one of the following predfined categories:
'History', 'Geography', 'Science', 'Technology', 'Mathematics', 'Literature', 'Art', 'Music', 'Film', 'Television', 'Sports', 'Politics', 'Philosophy', 'Religion', 'Sociology', 'Psychology', 'Economics', 'Business', 'Medicine', 'Biology', 'Chemistry', 'Physics', 'Astronomy', 'Environmental Science', 'Engineering', 'Computer Science', 'Linguistics', 'Anthropology', 'Archaeology', 'Education', 'Law', 'Military', 'Architecture', 'Fashion', 'Cuisine', 'Travel', 'Mythology', 'Folklore', 'Biography', 'Mythology', 'Social Issues', 'Human Rights', 'Technology Ethics', 'Climate Change', 'Conservation', 'Urban Studies', 'Demographics', 'Journalism', 'Cryptocurrency', 'Artificial Intelligence'
you will output a json object containing the following information:

{{
    "category": string // category name based on the article,
}}

Keep category names simple and use only lower case letters.
Articles can have only one category.
Article: {article}
    """

print(template)

Your task is to assess the article and categorize the article into one of the following predfined categories:
'History', 'Geography', 'Science', 'Technology', 'Mathematics', 'Literature', 'Art', 'Music', 'Film', 'Television', 'Sports', 'Politics', 'Philosophy', 'Religion', 'Sociology', 'Psychology', 'Economics', 'Business', 'Medicine', 'Biology', 'Chemistry', 'Physics', 'Astronomy', 'Environmental Science', 'Engineering', 'Computer Science', 'Linguistics', 'Anthropology', 'Archaeology', 'Education', 'Law', 'Military', 'Architecture', 'Fashion', 'Cuisine', 'Travel', 'Mythology', 'Folklore', 'Biography', 'Mythology', 'Social Issues', 'Human Rights', 'Technology Ethics', 'Climate Change', 'Conservation', 'Urban Studies', 'Demographics', 'Journalism', 'Cryptocurrency', 'Artificial Intelligence'
you will output a json object containing the following information:

{{
    "category": string // category name based on the article,
}}

Keep category names simple and use only lower case letters.


In [15]:
prompt = PromptTemplate(
    template=template,
    input_variables=["article"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

### Build llm chain

In [16]:
chain = prompt | llm | parser

### Cost estimation

In [69]:
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
system_prompt_token_count = len(encoding.encode(template))
system_prompt_token_count

266

In [70]:
system_prompt_token_count *= len(articles)
system_prompt_token_count

2660000

In [71]:
user_prompt_token_count = 0
for article in articles:
    user_prompt_token_count += len(encoding.encode(article))
user_prompt_token_count

739532

In [72]:
total_tokens = system_prompt_token_count + user_prompt_token_count
total_tokens

3399532

In [73]:
GPT4o_COST = 5
GPT4_COST = 10
GPT3_5_COST = 1.5

print("GPT4o Cost: ", (total_tokens * GPT4o_COST ) / 1000000)
print("GPT4 Cost: ", (total_tokens * GPT4_COST ) / 1000000)
print("GPT3.5 Cost: ", (total_tokens * GPT3_5_COST ) / 1000000)

GPT4o Cost:  16.99766
GPT4 Cost:  33.99532
GPT3.5 Cost:  5.099298


### Run inference

In [23]:
results = []
for article in tqdm(articles):
    try:
        result = chain.invoke({"article": article})
        print(result)
        if "Category" in result:
            results.append(result["Category"])
        elif "category" in result:
            results.append(result["category"])
        else:
            results.append("")
    except Exception as e:
        print("Exception Occured", e)
        results.append("")

  1%|█▋                                                                                                                                                                           | 1/100 [00:00<01:23,  1.19it/s]

{'category': 'politics'}



  2%|███▍                                                                                                                                                                         | 2/100 [00:01<01:07,  1.46it/s]

{'category': 'physics'}



  3%|█████▏                                                                                                                                                                       | 3/100 [00:02<01:03,  1.53it/s]

...

{'category': 'Sports'}



 99%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎ | 99/100 [01:04<00:00,  1.42it/s]

{'category': 'art'}


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [01:04<00:00,  1.54it/s]

{'category': 'Biology'}


### Write output as huggingface dataset

In [25]:
ids = dataset["train"][:len(results)]["id"]

In [30]:
df = pd.DataFrame({"id": ids, "category": results})
df = df[df["category"] != '']
df.to_csv("wiki-category-5k.csv", index=False)

,id,category
0,12,politics
1,39,physics
2,290,Linguistics
3,303,geography
4,305,mythology
...,...,...
95,746,geography
96,748,astronomy
97,751,Sports
98,752,art


In [44]:
category_dataset = Dataset.from_csv("wiki-category-5k.csv")

In [ ]:
category_dataset.push_to_hub("<HF-USERNAME>/<DATASET-NAME>", private=True)